In [ ]:
from __future__ import print_function
from IPython.display import display
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

import reentry
reentry.manager.scan()
    
from aiidalab_widgets_base import StructureUploadWidget, CodeDropdown
%matplotlib notebook

## Step 1: Select Structure

Before uploading, please
 * **Desolvate** the structure
 * Use **P1 symmetry**

In [ ]:
widget = StructureUploadWidget(node_class='CifData')
display(widget)

## Step 2: Select executable

In [ ]:
dropdown = CodeDropdown(input_plugin='zeopp.network')
display(dropdown)

## Step 3: Compute 

In [ ]:
import ipywidgets as ipw
from aiida_zeopp.workflows import ZeoppBlockPocketsWorkChain
from aiida.orm.data.base import Float
from aiida.work.run import run

probe_radius = ipw.BoundedFloatText(
    description='Probe radius [Angstrom]:', 
    min=0.1,
    max=100.0,
    value=1.8)

def on_click_submit(b):
    structure = widget.structure_node

    if structure is False:
        message.value = "Please upload a structure first"
        return None
        
    code = dropdown.selected_code
    if code is False:
        message.value = "Please select a code first"
        return None

    message.value = 'Please wait, pore properties are being computed.'
    btn_submit.disabled = True

    options = {
        "resources": {
            "num_machines": 1,
            "tot_num_mpiprocs": 1,
            "num_mpiprocs_per_machine": 1,
        },
        "max_wallclock_seconds": 5 * 60 * 60,
        "withmpi": False,
    }

    with submit_out:
        outputs = run(
            ZeoppBlockPocketsWorkChain,
            probe_radius=Float(probe_radius.value),
            structure=structure,
            zeopp_code=code
        )


btn_submit = ipw.Button(description='Compute Properties')
btn_submit.on_click(on_click_submit)
message = ipw.HTML('')
submit_out = ipw.Output()


display(
    probe_radius,
    btn_submit,
    submit_out,
    message
)